In [ ]:
#!pip install -q ultralytics clearml

In [9]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml


In [ ]:
import datetime
import os

from clearml import Dataset, Task
from google.colab import userdata
from ultralytics import YOLO


In [13]:
os.environ['CLEARML_API_ACCESS_KEY'] = userdata.get('CLEARML_API_ACCESS_KEY')
os.environ['CLEARML_API_SECRET_KEY'] = userdata.get('CLEARML_API_SECRET_KEY')

In [14]:
CLEARML_PROJECT_NAME = "CMT318-Object-Detection"
CLEARML_DATASET_NAME = "VOC2012_valsplit"
MODEL_NAME = "yolov8x"
AUTHOR = "syedhussain9655@gmail.com"
PLATFORM = 'colab'

In [ ]:
import random
import numpy as np
import torch
os.environ["PYTHONHASHSEED"] = str(42)

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
dataset = Dataset.get(
    dataset_name= CLEARML_DATASET_NAME,
    dataset_project= CLEARML_PROJECT_NAME
)
data_dir = dataset.get_mutable_local_copy("voc_yolo")

#Important Instructions to run on Google chrome
# Open /content/voc_yolo/data.yaml
# On line 22:
#   Change path value /content/voc_yolo

████████████████████████████▊  99% | 320.00/322.48 MB [00:05<00:00, 59.34MB/s]: /usr/local/lib/python3.11/dist-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
█████████████████████████████ 100% | 322.48/322.48 MB [00:05<00:00, 61.15MB/s]: 
█████████████████████████████ 100% | 321.83/321.83 MB [00:05<00:00, 61.25MB/s]: 


In [17]:
train_config = {
    'epochs': 200,
    'batch': 16,
    'imgsz': 640,
    'optimizer': 'SGD',
    'lr0': 0.01,
    'lrf': 0.2,
    'cos_lr': True,
    'momentum': 0.937,
    'weight_decay': 5e-4,
    'warmup_epochs': 5,
    'augment': True,
    'mosaic': 1.0,
    'mixup': 0.5,
    'dropout': 0.3,
    'multi_scale': True,
    'patience': 20,
    'device': 'cuda:0',
    'pretrained': False
}

model_name = "yolov8n"
author = "syedhussain9655@gmail.com"
platform = 'colab'

experiment_tags = [
    f"model_name:{model_name}",
    f"dataset:{CLEARML_DATASET_NAME}",
    f"platform:{platform}",
    f"author:{author}",
]

experiment_name = f"{model_name}_{CLEARML_DATASET_NAME}"

In [18]:
task = Task.init(
    project_name = CLEARML_PROJECT_NAME,
    task_name    = experiment_name,
    tags         = experiment_tags
)

task.connect(train_config)

ClearML Task: created new task id=0b971bf6c1424c83a2c87d623e8bb52e
ClearML results page: https://app.clear.ml/projects/31ab205b5fdb489d9ad1b4ed44a65563/experiments/0b971bf6c1424c83a2c87d623e8bb52e/output/log


{'epochs': 200,
 'batch': 16,
 'imgsz': 640,
 'optimizer': 'SGD',
 'lr0': 0.01,
 'lrf': 0.2,
 'cos_lr': True,
 'momentum': 0.937,
 'weight_decay': 0.0005,
 'warmup_epochs': 5,
 'augment': True,
 'mosaic': 1.0,
 'mixup': 0.5,
 'dropout': 0.3,
 'multi_scale': True,
 'patience': 20,
 'device': 'cuda:0',
 'pretrained': False}

In [19]:
!yolo settings tensorboard=True

JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [20]:
model = YOLO(f'{model_name}.yaml')

In [ ]:
CONFIG_YAML = os.path.join(data_dir,"data.yaml")

model.train(
    data=str(CONFIG_YAML),
    task='detect',
    epochs=train_config['epochs'],
    batch=train_config['batch'],
    imgsz=train_config['imgsz'],
    optimizer=train_config['optimizer'],
    lr0=train_config['lr0'],
    lrf=train_config['lrf'],
    cos_lr=train_config['cos_lr'],
    momentum=train_config['momentum'],
    weight_decay=train_config['weight_decay'],
    warmup_epochs=train_config['warmup_epochs'],
    augment=train_config['augment'],
    mosaic=train_config['mosaic'],
    mixup=train_config['mixup'],
    dropout=train_config['dropout'],
    multi_scale=train_config['multi_scale'],
    patience=train_config['patience'],
    device=train_config['device'],
    pretrained=train_config['pretrained'],
    exist_ok=True,
    save_period=5
)

Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/voc_yolo/data.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=True, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

100%|██████████| 755k/755k [00:00<00:00, 27.0MB/s]

Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

YOLOv8n summary: 129 layers, 3,014,748 parameters, 3,014,732 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 98.3MB/s]


...

      1/200      6.07G      3.394      4.675      3.954         12        768: 100%|██████████| 292/292 [02:14<00:00,  2.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:13<00:00,  2.77it/s]


                   all       1165       2747    0.00143     0.0757    0.00098    0.00031

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/200       6.1G      3.411       4.51      3.598         20        960: 100%|██████████| 292/292 [02:01<00:00,  2.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:11<00:00,  3.25it/s]


                   all       1165       2747    0.00217     0.0656    0.00175   0.000408

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/200       6.1G      3.118      4.376      3.321        146        608:  56%|█████▌    | 164/292 [01:06<01:23,  1.54it/s]